Script for preprocessing of EEG data for InstrAct EEG
Author: Ivan Padezhki

In [ ]:
import os
import os.path as op

import numpy as np

import mne
from mne_bids import BIDSPath, read_raw_bids
from mne.preprocessing import ICA

In [ ]:
# Get working tree and eeg files for the project
bids_root = "/mnt/NeuroNas/ivan/InstrAct_EEG/data"
datatype = "eeg"
bids_path = BIDSPath(root=bids_root, datatype=datatype)
print(bids_path)

In [ ]:
# Prepare for reading the data for the participant
task = "instract"
suffix = "eeg"
sub_number = "31"
bids_path = bids_path.update(subject=sub_number, task=task, suffix=suffix)

In [ ]:
# Read data file
raw = read_raw_bids(bids_path=bids_path, verbose=False)

### First look and visual annotation of muscle artifacts

In [ ]:
# Plot events
events, event_id = mne.events_from_annotations(raw, verbose=False)
fig = mne.viz.plot_events(events, sfreq=raw.info["sfreq"], first_samp=raw.first_samp, event_id=event_id)

In [ ]:
# Look at power line noise
fig = raw.compute_psd(tmax=np.inf, fmax=250).plot(
    average=True, picks="data", exclude="bads"
)

Begin with copying raw and annotating breaks and EMG artifacts


In [ ]:
# Break annotation 
break_annots = mne.preprocessing.annotate_break(
    raw=raw,
    min_break_duration=10, 
    t_start_after_previous=2,
    t_stop_before_next=2
)
raw.set_annotations(raw.annotations + break_annots)

In [ ]:
# Here, we open the raw file and manually annotate muscular artifacts
fig = raw.plot()
fig.fake_keypress("a")

In [ ]:
# Store annotations in a new variable 
interactive_annot = raw.annotations
# Save the annotations to a file
raw.annotations.save(f"sub-{sub_number}_saved-annotations.csv", overwrite=True)
# Set new annot to a copy of the data
raw_annotated = raw.copy().set_annotations(interactive_annot)

### If I am reloading previously annotated data, load the annotations here:

In [ ]:
annot_from_file = mne.read_annotations(f"/mnt/NeuroNas/ivan/InstrAct_EEG/data/derivatives/sub-{sub_number}/sub-{sub_number}_saved-annotations.csv")
raw_annotated = raw.copy().set_annotations(annot_from_file)

### Bad channel inspection and interpolation
 - use an ERP plot to see if bad channels can be spotted

In [ ]:
# Check if you annotated some channels as bad during the visual inspection
raw_annotated.info['bads']

In [ ]:
# Have a look at topoplots to see if any channels draw attention
events, event_dict = mne.events_from_annotations(raw_annotated)
epochs = mne.Epochs(
    raw_annotated,
    events,
    event_id=event_dict,
    tmin=-0.5,
    tmax=1,
    reject_by_annotation=True, # default parameter
)['encoding'].average().plot_topomap(show_names=True, size=3)

In [ ]:
# If you find some channels that are bad, you can mark them as such
raw_annotated.info['bads'] = ['T8', 'PO4', 'TP8', 'CP3', 'POz']
# And then you can plot the epochs again to see if the channels are still problematic
epochs = mne.Epochs(
    raw_annotated,
    events,
    event_id=event_dict,
    tmin=-0.5,
    tmax=1,
    reject_by_annotation=True, # default parameter
)['encoding'].average().plot()

In [ ]:
# Using spherical spline interpolation, interpolate the channels selected as bad
raw_interp = raw_annotated.load_data().copy().interpolate_bads(reset_bads=True)
# Plot the Evoked data to see if the bad channels have been successfully interpolated
events, event_dict = mne.events_from_annotations(raw_interp)
epochs = mne.Epochs(
    raw_interp,
    events,
    event_id=event_dict,
    tmin=-0.5,
    tmax=1,
    reject_by_annotation=True, # default parameter
)['encoding'].average().plot()

### Apply filters: 
low pass: 60Hz
high pass: 0.1 or 0.5

In [ ]:
# Depending on whether we've interpolated the bad channels or not, we need to apply the filter to the appropriate data
try: 
    filt_raw = raw_interp.load_data().filter(l_freq=0.5, h_freq=60.0)
except NameError: 
    filt_raw = raw_annotated.load_data().filter(l_freq=0.5, h_freq=60.0)

In [ ]:
# Have a look at the Evoked potential during experimental event (here Encoding) to check if the filtering has worked
events, event_dict = mne.events_from_annotations(filt_raw)
epochs = mne.Epochs(
    filt_raw,
    events,
    event_id=event_dict,
    tmin=-0.5,
    tmax=1,
    reject_by_annotation=True, # default parameter
)['encoding'].average().plot()

### ICA

Run Independent Component Analysis for artifacts such as blinks on already annotated and filtered data

In [ ]:
# Define and fit ICA
ica = ICA(n_components=20, max_iter="auto", random_state=42)
ica.fit(filt_raw)
ica

In [ ]:
# Open an interactive window to browse the ICs and have a look if they match the blinks and movements in the raw data
ica.plot_sources(filt_raw, show_scrollbars=False)

In [ ]:
# Have a look at topoplots of the ICA components - blinks are frontal and eye movements are lateralized in the front like a dipole
ica.plot_components()

In [ ]:
# Select ICA components that are suspected to reflect blinks and movements - change component numbers as needed
components = [2, 3]
# Have a look if the patterns make sense in the time course of the experiment (e.g. not concentrated in 1 moment only)
ica.plot_overlay(filt_raw, exclude=components)
ica.plot_properties(filt_raw, picks=components)

In [ ]:
# Apply ICA to the data, coppying the raw data first to compare if it worked
ica.exclude = components
reconst_raw = filt_raw.copy()
ica.apply(reconst_raw)

In [ ]:
# Chech if ICA has been applied correctly to all artefacts, and if the data looks clean
reconst_raw.plot()

Save filtered data and epochs

In [ ]:
# Save file to derivative folder
reconst_raw.save(f"/mnt/NeuroNas/ivan/InstrAct_EEG/data/derivatives/sub-{sub_number}/sub-{sub_number}_fit_ica.fif", overwrite=False)